In [69]:
import requests
import urllib
import math
import numpy as np
from PIL import Image
# 150 meters
SEARCH_RADIUS = 2500
IMAGE_SIZE = 64
ROAD_COLOURS = {
    "primary": (255, 0, 0),
    "secondary": (0, 255, 0),
    "tertiary": (0, 0, 255),
}

In [91]:
geocode = (43.698858, -79.436116)

query = urllib.parse.quote(f"""
[out:json][timeout:25];
nwr(around:{SEARCH_RADIUS}, {geocode[0]}, {geocode[1]})[highway~"^(primary|secondary|tertiary|motorway|motorway_link|residential)$"];
out geom;
""")
result = requests.get(f"https://overpass-api.de/api/interpreter?data={query}").json()

In [92]:
roads = {"primary":[], "secondary":[], "tertiary":[]}

for element in result["elements"]:
    if (element["tags"]["highway"] in "motorway motorway_link primary"):
        roads["primary"].append(element)
    elif (element["tags"]["highway"] in "secondary tertiary"):
        roads["secondary"].append(element)
    elif (element["tags"]["highway"] in "residential"):
        roads["tertiary"].append(element)

In [93]:
def geo_distance(coord1, coord2):
    distance_lat = math.radians(coord2[0] - coord1[0])
    distance_lon = math.radians(coord2[1] - coord1[1])
    angle = (
        math.sin(distance_lat/2) * math.sin(distance_lat/2) +
        math.cos(math.radians(coord1[0])) * math.cos(math.radians(coord2[0])) *
        math.sin(distance_lon/2) * math.sin(distance_lon/2)
    )
    curve = 2 * math.atan2(math.sqrt(angle), math.sqrt(1-angle))
    return  6378 * curve * 1000

# moves a geocode by offset (n meters, m meters)
def move_geocode(geocode, offset):
    return (geocode[0] + offset[0]/111111, geocode[1] + offset[1]/(math.cos(math.radians(geocode[0]))*111111))

# get the offset in meters of the two geocodes
def geocode_offset(origin, geocode):
    return ((geocode[0]-origin[0])*111111, (geocode[1]-origin[1])*111111*math.cos(math.radians(geocode[0])))

In [94]:
# draw roads to image
def dist(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
def floor_point(p1):
    return (math.floor(p1[0]), math.floor(p1[1]))
def draw_point(arr, p1, color):
    if p1[0] >= 0 and p1[1] >= 0 and p1[0] < len(arr) and p1[1] < len(arr[0]):
        for i in range(len(color)):
            arr[p1[0]][p1[1]][i] += color[i]
def draw_line(arr, points, color):
    cur_point = points.pop(0)
    next_point = points.pop(0)
    last_draw = (-1, -1)
    draw_point(arr, floor_point(cur_point), color)
    while True:
        if dist(last_draw, floor_point(next_point)) <= 1:
            if len(points) == 0:
                break
            cur_point = next_point
            next_point = points.pop(0)
            continue
        cur_draw = floor_point(cur_point)
        if dist(cur_draw, last_draw) != 0:
            draw_point(arr, cur_draw, color)
            last_draw = cur_draw
        diff_vec = (next_point[0]-cur_point[0], next_point[1]-cur_point[1])
        mag = math.sqrt(diff_vec[0]**2  + diff_vec[1]**2)
        diff_vec = (diff_vec[0]/mag, diff_vec[1]/mag)
        cur_point = (cur_point[0] + diff_vec[0], cur_point[1] + diff_vec[1])

im_arr = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3))

for roadtype in roads.keys():
    for road in roads[roadtype]:
        points = []
        for point in road["geometry"]:
            offset = geocode_offset(geocode, (point["lat"], point["lon"]))
            points.append((offset[0]*IMAGE_SIZE/SEARCH_RADIUS/1.5+IMAGE_SIZE/2, offset[1]*IMAGE_SIZE/SEARCH_RADIUS/1.5+IMAGE_SIZE/2))
        draw_line(im_arr, points, ROAD_COLOURS[roadtype])

# draw_line(im_arr, [(0,0),(10,40),(64,64),(20,50)], (0,0,255))

im_arr = im_arr.astype('uint8')
im = Image.fromarray(im_arr, mode="RGB")
im.save("image.png")